In [1]:
!pip install gym

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 721.7/721.7 kB 2.8 MB/s  0:00:0036m-:--:--
  Installing build dependencies ... one
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for gym: filename=gym-0.26.2-py3-none-any.whl size=827730 sha256=fa390e77f09f1787957c600199659c3a71dc96138d21e3dd272fb3e698f361cb
  Stored in directory: /Users/uliamalueva/Library/Caches/pip/wheels/95/51/6c/9bb05ebbe7c5cb8171dfaa3611f32622ca4658d53f31c79077
Successfully built gym
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [gym]


In [2]:
# не забудьте установить gym, если он у вас ещё не установлен
# при помощи pip install gym

import gym
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# фиксируем сид
np.random.seed(17)
tf.random.set_seed(17)

Gym has been unmaintained since 2022 and does not support NumPy 2.0 amongst other critical functionality.
Please upgrade to Gymnasium, the maintained drop-in replacement of Gym, or contact the authors of your software and request that they upgrade.
Users of this version of Gym should be able to simply replace 'import gym' with 'import gymnasium as gym' in the vast majority of cases.
See the migration guide at https://gymnasium.farama.org/introduction/migration_guide/ for additional information.


In [4]:
env = gym.make("CartPole-v0")  # создадим среду
env.reset(seed=17)

(array([ 0.03450748, -0.03390269,  0.00577445, -0.01319201], dtype=float32),
 {})

In [5]:
env.observation_space

Box([-4.8000002e+00 -3.4028235e+38 -4.1887903e-01 -3.4028235e+38], [4.8000002e+00 3.4028235e+38 4.1887903e-01 3.4028235e+38], (4,), float32)

In [6]:
env.action_space

Discrete(2)

In [10]:
num_actions = env.action_space.n
state_shape = env.observation_space.shape

inputs = layers.Input(shape=(4,))
common = layers.Dense(64, activation="relu")(inputs)
common2 = layers.Dense(64, activation="relu")(common)
action = layers.Dense(2, activation="softmax")(common2)
critic = layers.Dense(1)(common2)

model = keras.Model(inputs=inputs, outputs=[action, critic])

In [11]:
# в качестве оптимизатора будем по стандарту использовать ADAM
optimizer = keras.optimizers.Adam(learning_rate=0.01)

# в качестве функции потерь - Huber loss
# она похожа на MSE, но оканчивается линейрой функцией, а не квадратичной
# это придаст чуть больше стабильности нашему процессу обучения
huber_loss = keras.losses.Huber()

In [13]:
# зададим discount-фактор для награды
gamma = 0.995

# как долго длится эпизод
max_steps_per_episode = 10000

# минимальная машинная точность
eps = np.finfo(np.float32).eps.item()

# выделим списки для хранения значений
# вероятности действий, критика и награды
action_probs_history = []
critic_value_history = []
rewards_history = []


running_reward = 0
episode_count = 0

while True:
    # сбрасываем состояние к изначальному и записываем его
    state = env.reset()
    # Убедимся, что state является numpy array правильной формы
    if isinstance(state, tuple):
        state = state[0]  # для новых версий gym
    state = np.array(state, dtype=np.float32)
    
    episode_reward = 0
    
    # проводим раунд игры, записывая работу нашей нейросети в GradientTape
    with tf.GradientTape() as tape:
        # идём по раунду шагами
        for timestep in range(1, max_steps_per_episode):
            env.render()  # для графического отображения окна с игрой

            state = tf.convert_to_tensor(state)
            state = tf.expand_dims(state, 0)

            # зная состояние, модель должна нам предсказать действие 
            # и поправку от критика
            action_probs, critic_value = model(state)
            
            # поправку критика сразу сохраним
            critic_value_history.append(critic_value[0, 0])

            # выберем действие, исходя из предсказанных вероятностей
            action = np.random.choice(num_actions, p=np.squeeze(action_probs))
            
            # вероятность выбранного действия сохраняем в историю
            # можно сразу и прологарифмировать, как того требует наш лосс
            action_probs_history.append(tf.math.log(action_probs[0, action]))

            # говорим среде выполнить выбранное действие
            # и получаем от неё следующее состояние и награду
            # если игра окончена, среда вернут done=True
            result = env.step(action)
            state, reward, done = result[0], result[1], result[2]

            # Убедимся, что state является numpy array
            state = np.array(state, dtype=np.float32)
            
            # записываем полученную награду
            rewards_history.append(reward)
            
            episode_reward += reward

            if done:
                break

        # пересчитываем running_reward с затуханием
        running_reward = 0.05 * episode_reward + (1 - 0.05) * running_reward

        # награда на каждом шаге игры (то, что нам и должен предсказать критик)
        # определяется как суммарная награда от этого и всех последующих шагов (с затуханием)
        # считаем их все в 1 проход и добавляем в returns
        returns = []
        discounted_sum = 0
        for r in rewards_history[::-1]:
            discounted_sum = r + gamma * discounted_sum
            returns.insert(0, discounted_sum)

        # нормализуем их для повышения стабильности обучения
        returns = np.array(returns)
        returns = (returns - np.mean(returns)) / (np.std(returns) + eps)
        returns = returns.tolist()

        # осталось посчитать значения функций потерь, чтобы затем осуществить шаг градиентного спуска
        history = zip(action_probs_history, critic_value_history, returns)
        actor_losses = []
        critic_losses = []
        for log_prob, value, ret in history:
            # на каждом шаге считаем лосс для actor и critic
            # у первого это просто награда (за вычетом поправки от критика), 
            # умноженная на логарифмическую вероятность действия
            diff = ret - value
            actor_losses.append(-log_prob * diff)  # actor loss

            # критик же оканчивается обычной регрессией:
            # он должен уметь предсказывать ожидаемую награду на каждом шаге
            critic_losses.append(
                loss(tf.expand_dims(value, 0), tf.expand_dims(ret, 0))
            )

        # теперь можем осуществить стандартный шаг градиентного спуска
        # чтобы запустить его из одной точки, сложим все лоссы
        # т.к. при суммировании градиент от каждого слагаемого отправится
        # только в направлении данного слагаемого, это то, что нужно
        loss_value = sum(actor_losses) + sum(critic_losses)
        grads = tape.gradient(loss_value, model.trainable_variables)
        optimizer.apply_gradients(zip(grads, model.trainable_variables))

        # перед следующим эпизодом очистим все списки
        action_probs_history.clear()
        critic_value_history.clear()
        rewards_history.clear()

    episode_count += 1
    
    # каждые 10 эпизодов будем выводить running reward
    if episode_count % 10 == 0:
        print(f"Episode: {episode_count}, running reward: {running_reward:.2f}")

    # если running reward стал больше 195, цель достигнута
    if running_reward > 195:
        print(f"Solved at episode {episode_count}!")
        break

# саму среду в конце нужно закрыть
# чтобы рендер прекратился и можно было дальше работать в питоне
env.close()

/opt/anaconda3/lib/python3.12/site-packages/gym/utils/passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):


Episode: 10, running reward: 11.19
Episode: 20, running reward: 12.13
Episode: 30, running reward: 12.69
Episode: 40, running reward: 13.54
Episode: 50, running reward: 15.33
Episode: 60, running reward: 14.85
Episode: 70, running reward: 15.48
Episode: 80, running reward: 26.67
Episode: 90, running reward: 28.87
Episode: 100, running reward: 37.05
Episode: 110, running reward: 38.04
Episode: 120, running reward: 40.40
Episode: 130, running reward: 48.90
Episode: 140, running reward: 63.83
Episode: 150, running reward: 83.46
Episode: 160, running reward: 136.74
Episode: 170, running reward: 103.49
Episode: 180, running reward: 66.92
Episode: 190, running reward: 44.75
Episode: 200, running reward: 31.13
Episode: 210, running reward: 22.42
Episode: 220, running reward: 17.37
Episode: 230, running reward: 14.39
Episode: 240, running reward: 12.89
Episode: 250, running reward: 11.79
Episode: 260, running reward: 11.25
Episode: 270, running reward: 11.18
Episode: 280, running reward: 11.00

In [17]:
# Сбросим состояние среды и выполним 1 шаг
state = env.reset()
# Для новых версий gym может возвращаться tuple, берем только состояние
if isinstance(state, tuple):
    state = state[0]
state = np.array(state, dtype=np.float32)

print("Исходное состояние:", state)

Исходное состояние: [-0.03200178  0.04104776 -0.00531698 -0.0443652 ]


In [18]:
# Преобразуем в тензор
state_tensor = tf.convert_to_tensor(state)
state_tensor = tf.expand_dims(state_tensor, 0)

print("Состояние как тензор:", state_tensor)

Состояние как тензор: tf.Tensor([[-0.03200178  0.04104776 -0.00531698 -0.0443652 ]], shape=(1, 4), dtype=float32)


In [19]:
# Получаем вероятности действий от модели
action_probs, _ = model(state_tensor)

print("\nВероятности действий:", action_probs.numpy())
print("Сумма вероятностей:", np.sum(action_probs.numpy()))


Вероятности действий: [[0.8601593  0.13984066]]
Сумма вероятностей: 0.99999994


In [20]:
# Посмотрим на конкретные значения
probs = action_probs.numpy()[0]
print(f"Вероятность действия 0: {probs[0]:.4f}")
print(f"Вероятность действия 1: {probs[1]:.4f}")

Вероятность действия 0: 0.8602
Вероятность действия 1: 0.1398
